## scipy version

In [70]:
from scipy.spatial.transform import Rotation
import numpy as np


def get_rotvec(triangle):
    # triangle = np.array(triangle, dtype=float)
    p1, p2, p3 = triangle
    normal = np.cross(p2 - p1, p3 - p1)
    normal /= (sum(normal ** 2)) ** 0.5 
    z_axis = np.array([0, 0, 1])
    angle = np.arccos(np.dot(normal, z_axis))
    axis = np.cross(normal, z_axis)
    if np.linalg.norm(axis) < 1e-6:
        axis = np.array([1, 0, 0])
    else:
        axis /= (sum(axis ** 2)) ** 0.5 
    return angle * axis


def rot_to_xy_plane(rotvec):
    rot1 = Rotation.from_rotvec(rotvec)
    return rot1

def rot_to_x_axis(triangle):
    p1, p2, p3 = triangle
    angle = np.arctan2(p2[1], p2[0])
    rot2 = Rotation.from_euler('z', -angle)
    return rot2

def get_quaternion_rotation(triangle):
    rotvec = get_rotvec(triangle)
    rot1 = rot_to_xy_plane(rotvec)

    # Rotate around z-axis to align second point with x-axis
    rotated_triangle = rot1.apply(triangle)
    # print(rot1.as_quat())
    # print(rot1.as_matrix())
    rot2 = rot_to_x_axis(rotated_triangle)

    # Combine the two rotations
    # print("scipy rot1_q:", rot1.as_quat())
    # print("scipy rot2_q:", rot2.as_quat())
    # print("scipy combined_q:", (rot2 * rot1).as_quat())
    return rot2 * rot1


def rotate_triangle_to_xy(triangle):
    # triangle = np.array(triangle, dtype=float)
    rot = get_quaternion_rotation(triangle)
    rotated_triangle = rot.apply(triangle)
    return rotated_triangle


# triangle = np.array([(0, 0, 0), (1, 2, 3), (4, 5, 6)], dtype=np.float64)
# rotated_triangle = rotate_triangle_to_xy(triangle)
# print(rotated_triangle)

## numpy version

In [295]:
_rotvec = np.array([1, 2, 3])
_rotvec[None, :][0, :]

array([1, 2, 3])

In [296]:
import numpy as np


# def from_rotvec_np(rotvec):
#     is_single = False
#     # rotvec = np.asarray(rotvec, dtype=float)

#     if rotvec.ndim not in [1, 2] or rotvec.shape[-1] != 3:
#         raise ValueError("Expected `rot_vec` to have shape (3,) "
#                          "or (N, 3), got {}".format(rotvec.shape))

#     if rotvec.shape == (3,):
#         crotvec = rotvec[None, :]
#         is_single = True
#     else:
#         crotvec = rotvec

#     num_rotations = crotvec.shape[0]
#     quat = np.zeros((num_rotations, 4))

#     for ind in range(num_rotations):

#         angle = (sum(crotvec[ind, :] ** 2)) ** 0.5

#         # if angle <= 1e-3:  # small angle
#         #     angle2 = angle * angle
#         #     scale = 0.5 - angle2 / 48 + angle2 * angle2 / 3840
#         # else:  # large angle
#         #     scale = np.sin(angle / 2) / angle
        
#         scale = np.sin(angle / 2) / angle

#         quat[ind, 0] = scale * crotvec[ind, 0]
#         quat[ind, 1] = scale * crotvec[ind, 1]
#         quat[ind, 2] = scale * crotvec[ind, 2]
#         quat[ind, 3] = np.cos(angle / 2)
    
#     if is_single:
#         return quat[0]
#     else:
#         return quat
    

def from_rotvec_np(rotvec):
    angle = (sum(rotvec ** 2)) ** 0.5
    scale = np.sin(angle / 2) / angle
    return np.array([scale * rotvec[0],
                     scale * rotvec[1],
                     scale * rotvec[2],
                     np.cos(angle / 2)])



def _quat_mul_np(q1, q2):
    x1, y1, z1, w1 = q1
    x2, y2, z2, w2 = q2
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 + y1 * w2 + z1 * x2 - x1 * z2
    z = w1 * z2 + z1 * w2 + x1 * y2 - y1 * x2
    return np.array([x, y, z, w])
# slower
# def _quat_mul_np2(q1, q2):
#     q1_matrix = np.array([[q1[3], -q1[2], q1[1], q1[0]], [q1[2], q1[3], -q1[0], q1[1]], [-q1[1], q1[0], q1[3], q1[2]], [-q1[0], -q1[1], -q1[2], q1[3]]])
#     return np.einsum('ij,j->i', q1_matrix, q2)


def quaternion_to_matrix_np(quaternion):
    x, y, z, w = quaternion
    R = [[1 - 2*(y**2 + z**2), 2*(x*y - z*w), 2*(x*z + y*w)],
         [2*(x*y + z*w), 1 - 2*(x**2 + z**2), 2*(y*z - x*w)],
         [2*(x*z - y*w), 2*(y*z + x*w), 1 - 2*(x**2 + y**2)]]
    return np.array(R, dtype=np.float64)

# # slower
# def quaternion_to_matrix_np2(quaternion):
#     """
#     Convert a quaternion to a rotation matrix using NumPy.
    
#     :param quaternion: A list or array-like object containing four elements representing a quaternion [x, y, z, w].
#     :return: A 3x3 NumPy array representing the rotation matrix.
#     """
#     q = np.outer(quaternion, quaternion)
#     diag_indices = np.diag_indices(3)
#     rotation_matrix = np.array(
#         [[q[1, 1]+q[2, 2], q[0, 1]-q[2, 3], q[0, 2]+q[1, 3]],
#          [q[0, 1]+q[2, 3], q[0, 0]+q[2, 2], q[1, 2]-q[0, 3]],
#          [q[0, 2]-q[1, 3], q[1, 2]+q[0, 3], q[0, 0]+q[1, 1]]],
#         dtype=np.float64)
#     rotation_matrix *= 2
#     rotation_matrix[diag_indices] *= -1
#     rotation_matrix[diag_indices] += 1
#     return rotation_matrix


# slower
# def from_euler_np(seq, angles):
#     if seq == 'z':
#         cy_val = np.cos(angles * 0.5)
#         sy_val = np.sin(angles * 0.5)
#         cp_val = 1.
#         sp_val = 0.
#         cr_val = 0.
#         sr_val = 1.

#         q = [cy_val * cp_val * sr_val - sy_val * sp_val * cr_val,
#              sy_val * cp_val * sr_val + cy_val * sp_val * cr_val,
#              sy_val * cp_val * cr_val - cy_val * sp_val * sr_val,
#              cy_val * cp_val * cr_val + sy_val * sp_val * sr_val][::-1]
#     # Add code for other sequences here
#     return q


def from_euler_np(seq, angles):
    if seq == 'z':
        cy_val = np.cos(angles * 0.5)
        sy_val = np.sin(angles * 0.5)
    return [0, 0, sy_val, cy_val]


def rot_to_x_axis_np(triangle):
    p1, p2, p3 = triangle
    angle = np.arctan2(p2[1], p2[0])
    rot2 = from_euler_np('z', -angle)
    return rot2


def get_quaternion_rotation_np(triangle):
    p1, p2, p3 = triangle
    normal = np.cross(p2 - p1, p3 - p1)
    normal /= (sum(normal ** 2)) ** 0.5 
    z_axis = np.array([0, 0, 1])
    angle = np.arccos(normal.dot(z_axis))
    axis = np.cross(normal, z_axis)
    # if (sum(axis ** 2)) ** 0.5 < 1e-6:
    #     axis = np.array([1, 0, 0])
    # else:
    #     axis /= (sum(axis ** 2)) ** 0.5 
    axis /= (sum(axis ** 2)) ** 0.5
    # print("axis", axis)
    # print("angle numpy", angle)
    rot1 = from_rotvec_np(angle * axis)
    # print(rot1)
    rot1_m = quaternion_to_matrix_np(rot1)
    # print("rot1_m", rot1_m)

    # Rotate around z-axis to align second point with x-axis
    rotated_triangle = triangle.dot(rot1_m.T)
    p1, p2, p3 = rotated_triangle
    angle = np.arctan2(p2[1], p2[0])
    rot2 = from_euler_np('z', -angle)
    # print("rot2", rot2)

    # Combine the two rotations
    return quaternion_to_matrix_np(_quat_mul_np(rot2, rot1))


def rotate_triangle_to_xy_np(triangle):
    rot = get_quaternion_rotation_np(triangle)
    # print("rot np", rot)
    rotated_triangle = triangle.dot(rot.T)
    return rotated_triangle

## sympy version

In [139]:
import numpy as np
import sympy as sp


def from_rotvec(rotvec):
    is_single = False
    # rotvec = np.asarray(rotvec, dtype=float)

    if rotvec.ndim not in [1, 2] or rotvec.shape[-1] != 3:
        raise ValueError("Expected `rot_vec` to have shape (3,) "
                         "or (N, 3), got {}".format(rotvec.shape))

    if rotvec.shape == (3,):
        crotvec = rotvec[None, :]
        is_single = True
    else:
        crotvec = rotvec

    num_rotations = crotvec.shape[0]
    quat = sp.Matrix.zeros(num_rotations, 4)

    for ind in range(num_rotations):

        angle = (sum(crotvec[ind, :] ** 2)) ** 0.5

        # if angle <= 1e-3:  # small angle
        #     angle2 = angle * angle
        #     scale = 0.5 - angle2 / 48 + angle2 * angle2 / 3840
        # else:  # large angle
        #     scale = np.sin(angle / 2) / angle
        
        scale = sp.sin(angle / 2) / angle

        quat[ind, 0] = scale * crotvec[ind, 0]
        quat[ind, 1] = scale * crotvec[ind, 1]
        quat[ind, 2] = scale * crotvec[ind, 2]
        quat[ind, 3] = sp.cos(angle / 2)
    
    return quat
    # if is_single:
    #     return quat[0]
    # else:
    #     return quat


def _quat_mul(q1, q2):
    x1, y1, z1, w1 = q1
    x2, y2, z2, w2 = q2
    w = w1 * w2 - x1 * x2 - y1 * y2 - z1 * z2
    x = w1 * x2 + x1 * w2 + y1 * z2 - z1 * y2
    y = w1 * y2 + y1 * w2 + z1 * x2 - x1 * z2
    z = w1 * z2 + z1 * w2 + x1 * y2 - y1 * x2
    return np.array([x, y, z, w])
    

def quaternion_to_matrix(quaternion):
    x, y, z, w = list(quaternion)
    R = [[1 - 2*(y**2 + z**2), 2*(x*y - z*w), 2*(x*z + y*w)],
         [2*(x*y + z*w), 1 - 2*(x**2 + z**2), 2*(y*z - x*w)],
         [2*(x*z - y*w), 2*(y*z + x*w), 1 - 2*(x**2 + y**2)]]
    return sp.Matrix(R)


# def from_euler(seq, angles):
#     if seq == 'z':
#         cy_val = sp.cos(angles * 0.5)
#         sy_val = sp.sin(angles * 0.5)
#         cp_val = 1.
#         sp_val = 0.
#         cr_val = 0.
#         sr_val = 1.

#         q = [cy_val * cp_val * sr_val - sy_val * sp_val * cr_val,
#              sy_val * cp_val * sr_val + cy_val * sp_val * cr_val,
#              sy_val * cp_val * cr_val - cy_val * sp_val * sr_val,
#              cy_val * cp_val * cr_val + sy_val * sp_val * sr_val][::-1]
#     # Add code for other sequences here
#     return q


def from_euler(seq, angles):
    if seq == 'z':
        cy_val = sp.cos(angles * 0.5)
        sy_val = sp.sin(angles * 0.5)
    return [0., 0., sy_val, cy_val]


def rot_to_x_axis_sympy(triangle):
    p1, p2, p3 = triangle
    angle = np.arctan2(p2[1], p2[0])
    rot2 = from_euler('z', -angle)
    return rot2


def get_quaternion_rotation_sym(triangle):
    p1, p2, p3 = triangle
    normal = np.cross(p2 - p1, p3 - p1)
    normal /= (sum(normal ** 2)) ** 0.5 
    z_axis = np.array([0, 0, 1])
    angle = sp.acos(sp.Matrix(normal).dot(sp.Matrix(z_axis)))
    axis = np.cross(normal, z_axis)
    # if (sum(axis ** 2)) ** 0.5 < 1e-6:
    #     axis = np.array([1, 0, 0])
    # else:
    #     axis /= (sum(axis ** 2)) ** 0.5 
    axis /= (sum(axis ** 2)) ** 0.5
    # print("axis", axis)
    # print("angle sympy ", angle)
    rot1 = from_rotvec(angle * axis)
    # print(rot1)
    rot1_m = quaternion_to_matrix(rot1)
    # print("rot1_m", rot1_m)

    # Rotate around z-axis to align second point with x-axis
    rotated_triangle = triangle.dot(rot1_m.T)
    p1, p2, p3 = rotated_triangle
    angle = sp.atan2(p2[1], p2[0])
    rot2 = from_euler('z', -angle)
    # print("rot2", rot2)

    # Combine the two rotations
    return quaternion_to_matrix(_quat_mul(rot2, rot1))


def rotate_triangle_to_xy_sym(triangle):
    rot = get_quaternion_rotation_sym(triangle)
    # print("rot sp", rot)
    rotated_triangle = triangle.dot(rot.T)
    return rotated_triangle

In [297]:
triangle = np.array([(0, 0, 0), (1, 2, 3), (4, 5, 6)], dtype=np.float64)
# get_quaternion_rotation_sym(triangle), get_quaternion_rotation(triangle).as_matrix()
rotate_triangle_to_xy_sym(triangle), rotate_triangle_to_xy_np(triangle)  #, rotate_triangle_to_xy(triangle)

(array([[0, 0, 0],
        [3.74165738677394, -1.33226762955019e-15, 0],
        [8.55235974119758, 1.96396101212393, 4.44089209850063e-16]],
       dtype=object),
 array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 3.74165739e+00, -1.11022302e-16, -2.22044605e-16],
        [ 8.55235974e+00,  1.96396101e+00, -4.44089210e-16]]))

In [299]:
%%timeit
rotate_triangle_to_xy(triangle)

241 µs ± 6.77 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [300]:
%%timeit
rotate_triangle_to_xy_np(triangle)

191 µs ± 2.69 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [301]:
%%timeit
rotate_triangle_to_xy_sym(triangle)

4.08 ms ± 37.7 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
%%timeit
get_quaternion_rotation(triangle).as_matrix()

207 µs ± 20.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [7]:
# [[ 0.00000000e+00  0.00000000e+00  0.00000000e+00]
#  [ 3.74165739e+00 -3.33066907e-16  2.22044605e-16]
#  [ 8.55235974e+00  1.96396101e+00  0.00000000e+00]]

In [8]:
m = get_quaternion_rotation(triangle).as_matrix()

In [9]:
triangle.dot(m.T)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.74165739e+00, -3.33066907e-16,  1.66533454e-16],
       [ 8.55235974e+00,  1.96396101e+00,  3.33066907e-16]])

In [10]:
from scipy.spatial.transform import Rotation as R
import numpy as np

r = R.from_quat([0, 0, np.sin(np.pi/4), np.cos(np.pi/4)])
r.as_matrix()

array([[ 2.22044605e-16, -1.00000000e+00,  0.00000000e+00],
       [ 1.00000000e+00,  2.22044605e-16,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  1.00000000e+00]])

In [11]:
v = [1, 2, 3]
v_rotated = r.apply(v)
v_rotated

array([-2.,  1.,  3.])

In [12]:
np.array(v).dot(r.as_matrix().T)

array([-2.,  1.,  3.])

In [13]:
r.as_matrix().dot(v)

array([-2.,  1.,  3.])

In [14]:
from phyloshape.utils import trans_vector_to_relative

In [15]:
from sympy import symbols

In [16]:
pl1 = symbols("x0:3")
pl2 = symbols("x3:6")
pl3 = symbols("x6:9")
vts = symbols("m0:3")

In [17]:
plane = np.array([pl1, pl2, pl3])

In [18]:
vect = np.array(vts)

In [19]:
relative_vet1 = trans_vector_to_relative(vect, plane)

In [20]:
qr = get_quaternion_rotation_sym(plane)
relative_vet2 = vect.dot(qr.T)

In [21]:
res1_func1 = sp.lambdify(pl1 + pl2 + pl3 + vts, relative_vet1[0])

In [22]:
res2_func1 = sp.lambdify(pl1 + pl2 + pl3 + vts, relative_vet2[0])

In [23]:
plane = np.array([[1,20,300],[4,5,6],[7,8,9]], dtype=float)
plane[1] -= plane[0]
plane[2] -= plane[0]
plane

array([[   1.,   20.,  300.],
       [   3.,  -15., -294.],
       [   6.,  -12., -291.]])

In [24]:
trans_vector_to_relative(np.array([10,11,13], dtype=float), plane)

array([-13.59082887,  14.31564872,  -0.59293523])

In [25]:
qr = get_quaternion_rotation_sym(plane)
relative_vet2 = np.array([10.,11.,13.]).dot(np.array(qr).T)
relative_vet2

array([-13.4416527457314, 14.4558084959468, -0.592935233969217],
      dtype=object)

In [26]:
qr = get_quaternion_rotation(plane)
qr.apply(np.array([10.,11.,13.]))

array([-13.44165275,  14.4558085 ,  -0.59293523])

In [27]:
%%timeit
res1_func1(1,20,300,3.,  -15., -294.,6.,  -12., -291.,10,11,13)

116 µs ± 2.25 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [28]:
%%timeit
res2_func1(1,20,300,3.,  -15., -294.,6.,  -12., -291.,10,11,13)

4.5 ms ± 226 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [29]:
%%timeit
trans_vector_to_relative(np.array([10,11,13]), np.array([[1,20,300],[4,5,6],[7,8,9]]))

57 µs ± 680 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [30]:
res1_func1(1,20,300,3.,  -15., -294.,6.,  -12., -291.,10,11,13)

-13.590828871125021

In [31]:
res2_func1(1,20,300,3.,  -15., -294.,6.,  -12., -291.,10,11,13)

-13.441652745731446

In [32]:
triangle = np.array([(0, 0, 0), (1, 2, 3), (4, 5, 6)], dtype=np.float64)

In [33]:
rotate_triangle_to_xy_sym(triangle)

Matrix([
[               0,                     0,                    0],
[3.74165738677394, -1.33226762955019e-15,                    0],
[8.55235974119758,      1.96396101212393, 4.44089209850063e-16]])

In [34]:
rotate_triangle_to_xy(triangle)

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
       [ 3.74165739e+00, -3.33066907e-16,  2.22044605e-16],
       [ 8.55235974e+00,  1.96396101e+00,  0.00000000e+00]])

In [35]:
plane = np.array([(0, 0, 0), (1, 2, 3), (4, 5, 6)], np.float64)

In [303]:
trans_vector_to_relative(np.array([1.1, 2.2, 3.3]), plane)

array([4.11582313e+00, 2.22044605e-16, 2.77555756e-16])

In [304]:
qr = get_quaternion_rotation(plane)
qr.apply(np.array([1.1, 2.2, 3.3]))

array([ 4.11582313e+00, -1.11022302e-16,  2.22044605e-16])

In [305]:
qr = get_quaternion_rotation_np(plane)
qr.dot(np.array([1.1, 2.2, 3.3]))

array([ 4.11582313e+00,  2.20154171e-17, -1.05116220e-16])

In [38]:
%%timeit
trans_vector_to_relative(np.array([1.1, 2.2, 3.3]), plane)

34.9 µs ± 915 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [39]:
%%timeit
qr = get_quaternion_rotation(plane)
qr.apply(np.array([1.1, 2.2, 3.3]))

242 µs ± 14.2 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [302]:
%%timeit
qr = get_quaternion_rotation_np(plane)
qr.dot(np.array([1.1, 2.2, 3.3]))

208 µs ± 31.7 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
